# Импортируем библиотеки

In [ ]:
import datetime, time
import random
import pyads
import psycopg as psycopg2
import time

## Тестовый коннект к серверу

In [27]:
import pyads
plc = pyads.Connection('192.168.1.177.1.1',801)

plc.open()

print(plc.read_by_name('MAIN.bSWITCH_1',pyads.PLCTYPE_BOOL))

plc.close()

True


In [ ]:
def db_execute(query,data):
    print(f'OK -----   {query} + ',data)

## Класс для работы с базой данных

In [ ]:
# тестовое подключение
TEST_DATA = (datetime.datetime.now(),datetime.datetime.now(),random.randint(0,100),'PLC.TEST_BOOL','admin')

Q_INSERT_IN_LOG = '''INSERT INTO process_flow_log
                    (
                        start_date, 
                        finish_date, 
                        event_duration, 
                        variable, 
                        user_name
                    )
                    VALUES (%s, %s, %s, %s, %s)'''

connection_info = "dbname=combiner_data user=postgres password=1122 host=localhost"
connection_to_db = psycopg2.connect(connection_info)
connection_to_db.execute(Q_INSERT_IN_LOG,TEST_DATA)
connection_to_db.commit()
connection_to_db.close()

In [54]:
class Database():

    DB_NAME = 'combiner_data'
    DB_USER = 'postgres'
    DB_PASS = '1122'
    DB_HOST = 'localhost'
    DB_ALT = 'CSV'              # Альтернативная DB. На счучай сбоя в подключении.

    DB_CONNECTION_STR = f'dbname={DB_NAME} user={DB_USER} password={DB_PASS} host={DB_HOST}'

    def __init__(self):
        self.connected = False
        self.db_connection = self.connect()


    def connect(self):
        connection = psycopg2.connect(self.DB_CONNECTION_STR)
        self.connected = True
        print('DB CONNECTION IS OPEN')
        return connection
    
    @property
    def cursor(self):
        return self.db_connection.cursor()

    def disconnect(self):
        self.db_connection.close()

    def send_query(self,query,data):
        if self.connected:
            self.cursor.execute(query,data) #TRY EXCEПТ
    
    def commit(self):
        self.db_connection.commit()
        self.cursor.close()

# if __name__ == "__main__":
#     start = datetime.datetime.now()
#     db = Database()
#     for i in range(60):
#         db.send_query(Q_INSERT_IN_LOG,TEST_DATA)
#     db.commit()
#     finish = datetime.datetime.now() -start
#     print(finish)
#     db.disconnect()


In [29]:
class Plc:
    def __init__(self,address,port,name='Unnamed'):
        self.address = address
        self.port = port
        self.plc_connection = self.connect()
        self._open_connection()
        self.test_state = False

    def connect(self):
        return pyads.Connection(self.address, self.port)
    
    def _open_connection(self):
        if self.plc_connection:
            self.plc_connection.open()
            print('PLC CONNECTION IS OPEN')

    def close_connection(self):
        if self.plc_connection.is_open:
            self.plc_connection.close()
            print('PLC CONNECTION IS CLOSE')
    
    def read_by_name(self,path,name):
        return self.plc_connection.read_by_name(path,name)

In [ ]:
class Variable:

    def __init__(self,path,var_type,plc,
                name='',ctrl_type = 'state', test = None):
        self.path = path
        self.name = name if name else path
        self.var_type = var_type
        self.plc = plc
        self.test = test

    @property
    def value(self):
        if self.test == True:
            return self.test
        else: 
            return self.plc.read_by_name(self.path,self.var_type)# read from plc


In [ ]:
class StateControler:

    QUERY = 'BASIC_QUERY'

    def __init__(self,var):
        self.var = var
        self.prev_state = self.var.value

    @property
    def change_detected(self):
        return self.var.value != self.prev_state
    
    def set_previous_state(self):
        self.prev_state = self.var.value

    def check_state(self):
        
        if self.change_detected:
            print('State is changed')
            self.execute_query()
            self.set_previous_state()

    def get_data(self):
        return 'Some DATA'

    def execute_query(self):
        if self.var.test:    
            db_execute(self.QUERY, self.get_data())
        else:
            database.send_query(self.QUERY, self.get_data())


In [ ]:
class SwitchControl(StateControler):

    QUERY = 'SWITCH_QUERY'

    last_change_time = ''

    @property
    def time_of_switch(self):
        return datetime.datetime.now()

    def get_data(self):
        name = self.var.name
        value = self.var.value
        timest = self.time_of_switch

        return (name,timest,value)

In [43]:
class SwitchEventControl(StateControler):
    
    QUERY = '''INSERT INTO process_flow_log
                    (
                        start_date, 
                        finish_date, 
                        event_duration, 
                        variable, 
                        user_name
                    )
                    VALUES (%s, %s, %s, %s, %s)'''


    def __init__(self, var, good_state ,condition=None):
        StateControler.__init__(self,var)
        self.condition = condition
        self.start_time = ''
        self.finish_time = ''
        self.flag = False
        self.good_state = good_state
        
    @property
    def change_detected(self):
        return self.var.value != self.good_state

    
    @property
    def duration(self):
        """ Возвращает длительность события - Tfinish - Tstart """
        duration = self.finish_time - self.start_time
        return duration.seconds


    def _set_time(self):
        if self.start_time:
            self.finish_time = datetime.datetime.now()
        else:
            self.start_time = datetime.datetime.now()


    def _set_flag(self):
        self.flag = not self.flag
        print(self.flag)


    def check_state(self):
        changes = self.change_detected 
        if not changes and self.flag:
            print('Завершаем Событие')
            self.finish_event()
            self._set_flag()
            self.execute_query()
        if changes and self.flag != True:
            print('Начинаем Событие')
            self.start_event()
            self._set_flag()


    def get_data(self):
        return (self.start_time,
                self.finish_time,
                self.duration,
                self.var.name,
                'PLACE_FOR_CONDITION')

    def start_event(self):
            self._set_time()

    def finish_event(self):
            self._set_time()

# if __name__ == '__main__':
    
#     import time

#     sec = SwitchEventControl(TEST_VAR,True)
#     TEST_VAR.test = True

#     sec.check_state()
   
#     #sec.var.plc.test_state = False
#     TEST_VAR.test = False
#     sec.check_state()
#     time.sleep(2)
#     TEST_VAR.test = False
#     sec.check_state()
#     database.commit()
#     database.disconnect()
        

In [56]:
database = Database()
plc = Plc('192.168.1.177.1.1',801)

var_c = Variable('MAIN.bSWITCH_1',pyads.PLCTYPE_BOOL,plc, name='Switch 1')
var_f = Variable('MAIN.bSWITCH_2',pyads.PLCTYPE_BOOL,plc, name='Switch 2')
var_d = Variable('MAIN.bSWITCH_3',pyads.PLCTYPE_BOOL,plc, name='Switch 3')
var_g = Variable('MAIN.bSWITCH_4',pyads.PLCTYPE_BOOL,plc, name='Switch 4')

s1_sc = SwitchEventControl(var_c,False)
s2_sc = SwitchEventControl(var_f,False)
s3_sc = SwitchEventControl(var_d,False)
s4_sc = SwitchEventControl(var_g,False)

while True:
    try:
        s1_sc.check_state()
        s2_sc.check_state()
        s3_sc.check_state()
        s4_sc.check_state()

        database.commit()
    except Exception as e:
        print(e)
        plc.close_connection()
        database.disconnect()
        break
    



DB CONNECTION IS OPEN
PLC CONNECTION IS OPEN
Начинаем Событие
True
Завершаем Событие
False
Начинаем Событие
True
Начинаем Событие
True
Начинаем Событие
True
Начинаем Событие
True
Завершаем Событие
False
Завершаем Событие
False
Завершаем Событие
False
ADSError: timeout elapsed (1861). 
PLC CONNECTION IS CLOSE


In [ ]:
TEST_VAR = Variable('MAIN.bSWITCH_1',pyads.PLCTYPE_BOOL,Plc('1'), name='Switch 1',test=False)

In [ ]:
# Подключаемся к контроллеру

plc = pyads.Connection('192.168.1.177.1.1',801)
plc.open()
plc.is_open
plc.read_by_name()


In [ ]:
import time

sec = SwitchEventControl(TEST_VAR)

sec.start_event()
print(sec.start_time)
time.sleep(5)
sec.finish_event()
print(sec.finish_time)
print()

In [ ]:
var_c = Variable('MAIN.bSWITCH_1',pyads.PLCTYPE_BOOL,plc, name='Switch 1')
var_f = Variable('MAIN.bSWITCH_2',pyads.PLCTYPE_BOOL,plc, name='Switch 2')
var_d = Variable('MAIN.bSWITCH_3',pyads.PLCTYPE_BOOL,plc, name='Switch 3')
var_g = Variable('MAIN.bSWITCH_4',pyads.PLCTYPE_BOOL,plc, name='Switch 4')

s1_sc = SwitchControl(var_c)
s2_sc = SwitchControl(var_f)
s3_sc = SwitchControl(var_d)
s4_sc = SwitchControl(var_g)

while True:
    s1_sc.check_state()
    s2_sc.check_state()
    s3_sc.check_state()
    s4_sc.check_state()

plc.close()



In [ ]:
plc.close()


In [ ]:
a = True
b = 4 if not a else a